### Import libraries

In [13]:
#Use with python/3.6.0 and tensorflow/1.10

import math
import sys
from sklearn.linear_model import LogisticRegression
import random
import os
import csv
import json
from collections import defaultdict
import gensim
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.linear_model import LogisticRegressionCV
import sklearn.metrics as metrics
from sklearn import metrics
from sklearn.utils import resample
from sklearn.model_selection import GridSearchCV
from glove import Corpus, Glove
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
import numpy as np
import scipy.spatial.distance as ds
from bilm import Batcher, BidirectionalLanguageModel, weight_layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import accuracy_score
from tensorflow.python.keras.constraints import maxnorm
from tensorflow.python.keras.callbacks import EarlyStopping
from sklearn.metrics import make_scorer, r2_score, mean_absolute_error, mean_squared_error

from bert_serving.client import BertClient
bc = BertClient()



### Load data, define target diagnosis

In [14]:
## change data path to where the mimic sequences are located
data_path = "/users/emily/Documents/mimic_seq/"
random.seed(1)

# list for files
train_files = []
valid_files = []
full_data_files = []

for i in range(7):
    train_files.append(data_path + 'test_'+str(i))
for i in range(7,10):
    valid_files.append(data_path + 'test_'+str(i))
for i in range(10):
    full_data_files.append(data_path + 'test_'+str(i))

    
target_diagnosis = ['d_041']
passed = target_diagnosis

### Take raw mimic sequence data and generate a list of medical concepts for each patient

In [19]:
sentences = []
count = -1
for i in train_files:
    with open(i) as f:
        for s in f:
            count = count + 1
            sentences.append(s.split("|")[2].split(" ") +
                             s.split("|")[3].replace("\n", "").split(" "))


### Identify the unique medical concepts in the data

In [20]:
import itertools
flat_sentences = list(itertools.chain(*sentences))
events = np.unique(flat_sentences)
events = np.ndarray.tolist(events)

### Generate embeddings dataframe

In [22]:
AUC_logistic_elmo = []
AUC_DL_elmo = []
F1_logistic_elmo = []
F1_DL_elmo = []
AUC_lasso_bert = []
F1_lasso_bert = []


for q in range(len(passed)):    
    list_duplicates = []
    next_diags = []
    final_dx_train = []
    prev_diags = []
    disease_prev = []
    count = -1
    
    ## Identify patients that have the target diagnosis in their history (so we can omit them)
    
    for i in train_files:
        with open(i) as f:
            for s in f:
                count = count + 1
                next_diags = s.split("|")[0].split(",")
                final_dx_train.append(next_diags)
                prev_diags = [e for e in s.split("|")[2].split(" ") if e.startswith("d_")]

                if passed[q] in prev_diags: # if disease exit in the previous diagnosis
                    
                    disease_prev.append(1)
                    
                else:
                    disease_prev.append(0)


    ## create a vector indicating which patients have disease as a final diagnosis
    disease_train = []
    for sublist in final_dx_train:
        if passed[q] in sublist:
            disease_train.append(1)
        else:
            disease_train.append(0)
    count = -1
    count2 = 0
    
    ## In each patient history, generate a contextual BERT encoding for each word, do a weighted average via temporal factor for patient history 
    
    patient_seq_all_disease = []
    exclude = []
    test_feed = []
    for i in train_files: 
        with open(i) as f:
            for line in f:
                count = count + 1
                patient_seq = []
                feed_events = line.split("|")[2].split(" ")
                

                if disease_prev[count] == 1: # Only include events before the target diagnosis
                    target = feed_events.index(passed[q]) 
                    feed_events = feed_events[:target]
                    if feed_events == []:
                        count2 = count2+1
                        exclude.append(1) #exclude patients that have no medical concepts before the target diagnosis
                    else:    
                        exclude.append(0)
                        te = len(feed_events)
                        weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(feed_events) if e in events]
                        sum_weights = sum(weight for event,weight in weighted_events)
                        new = []
                        new_feed_events = []
                        vec = []

                        bigvec = []
                        newfeed = []
                        patient_seq = []
                        for n in range(len(feed_events)):
                            newfeed.append(feed_events[n].replace("_",r"\u"))
                        feeds = (newfeed)
                        new = ' '.join(feeds)
                        new_feed_events = [new]
                        texts2 = [x.split() for x in new_feed_events]              
                        vec = (bc.encode(texts2, is_tokenized=True))
                        vec = (vec[0][1:len(feed_events)+1])
                        for a in weighted_events:
                            event, weight = a
                            patient_seq.append(weight*vec[feed_events.index(event)])
                        patient_seq_all_disease.append(sum(patient_seq)/sum_weights)


                else: 
                    exclude.append(0)
                    te = len(feed_events)
                    weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(feed_events) if e in events]
                    sum_weights = sum(weight for event,weight in weighted_events)
                    new = []
                    new_feed_events = []
                    texts2 = []
                    vec = []

                    bigvec = []
                    newfeed = []
                    patient_seq = []
                    for n in range(len(feed_events)):
                        newfeed.append(feed_events[n].replace("_",r"\u"))
                    feeds = (newfeed)
                    new = ' '.join(feeds)
                    new_feed_events = [new]
                    texts2 = [x.split() for x in new_feed_events]              
                    vec = (bc.encode(texts2, is_tokenized=True))
                    vec = (vec[0][1:len(feed_events)+1])
                    for a in weighted_events:
                        event, weight = a
                        patient_seq.append(weight*vec[feed_events.index(event)])
                    patient_seq_all_disease.append(sum(patient_seq)/sum_weights)

    ## Create a dataframe for all the patient vectors
    data_disease=pd.DataFrame(patient_seq_all_disease)

    ## Create the train labels, make sure to get rid of labels where we exclude patients
    
    a = 0
    disease_final = []
    for i in range(0,len(exclude)):
        if exclude[i] == 0:
            disease_final.append(disease_train[i])
        else:
            a = a+1


    data_disease[passed[q]] = disease_final


    X_train_disease=data_disease.iloc[:,0:100]
    Y_train_disease =data_disease.iloc[:,100]



   

    ## For valid files, create a vector with previous diagnoses and note when disease in previous diagnoses, similar to what we did for train data   
    dis_prev = []
    pt_count = 0
    demograph = []
    for i in valid_files: 
        with open(i) as f:
            for line in f:
                patient_seq_test = []
                prev_diags = [e for e in line.split("|")[2].split(" ") if e.startswith("d_")]
                next_diags = line.split("|")[0].split(",")
                demograph.append(line.split("|")[1].split(","))
                if passed[q] in prev_diags: # if disease exit in the previous diagnosis
                    
                    dis_prev.append(1)
                    
                else:
                    dis_prev.append(0)

   
    #create a vector of all final diagnoses
    final_dx_test = []
    for i in valid_files:
        with open(i) as f:
            for s in f:
                final_dx_test.append(s.split("|")[0].split(","))

    #create a vector when disease in final diagnoses
    count = -1
    disease_test = []
    for sublist in final_dx_test:
        count = count + 1
        if passed[q] in sublist:
            disease_test.append(1)
        else:
            disease_test.append(0)

    exclude=[]

    ##create full testing dataframe with embeddings                
    count = -1
    count2 = 0
   
    patient_seq_all_disease_test = []
    exclude = []
    for i in valid_files: 
        with open(i) as f:
            for line in f:
                count = count + 1

                patient_seq = []
                feed_events = line.split("|")[2].split(" ")

                if dis_prev[count] == 1:
                    target = feed_events.index(passed[q])
                    feed_events = feed_events[:target]
                    if feed_events == []:
                        count2 = count2+1
                        exclude.append(1)
                    else:

                        exclude.append(0)
                        te = len(feed_events)
                        weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(feed_events) if e in events]
                        sum_weights = sum(weight for event,weight in weighted_events)
                        new = []
                        new_feed_events = []
                        texts2 = []
                        vec = []

                        bigvec = []
                        newfeed = []
                        patient_seq = []
                        for n in range(len(feed_events)):
                            newfeed.append(feed_events[n].replace("_",r"\u"))
                        feeds = (newfeed)
                        new = ' '.join(feeds)
                        new_feed_events = [new]
                        texts2 = [x.split() for x in new_feed_events]              
                        vec = (bc.encode(texts2, is_tokenized=True))
                        vec = (vec[0][1:len(feed_events)+1])
                        for a in weighted_events:
                            event, weight = a
                            patient_seq.append(weight*vec[feed_events.index(event)])
                        patient_seq_all_disease_test.append(sum(patient_seq)/sum_weights)

                else:

                    exclude.append(0)
                    te = len(feed_events)
                    weighted_events = [(e,  math.exp(decay*(j-te+1)/te)) for j, e in enumerate(feed_events) if e in events]
                    sum_weights = sum(weight for event,weight in weighted_events)
                    new = []
                    new_feed_events = []
                    texts2 = []
                    vec = []

                    bigvec = []
                    newfeed = []
                    patient_seq = []
                    for n in range(len(feed_events)):
                        newfeed.append(feed_events[n].replace("_",r"\u"))
                    feeds = (newfeed)
                    new = ' '.join(feeds)
                    new_feed_events = [new]
                    texts2 = [x.split() for x in new_feed_events]              
                    vec = (bc.encode(texts2, is_tokenized=True))
                    vec = (vec[0][1:len(feed_events)+1])
                    for a in weighted_events:
                        event, weight = a
                        patient_seq.append(weight*vec[feed_events.index(event)])
                    patient_seq_all_disease_test.append(sum(patient_seq)/sum_weights)


 

    ## Similar to training data, create a dataframe with labels and omit patients that are excluded.
    data_disease_test = pd.DataFrame(patient_seq_all_disease_test)

    disease_test_final = []
    for i in range(0,len(exclude)):
        if exclude[i] == 0:
            disease_test_final.append(disease_test[i])
        else:
            continue



    data_disease_test[passed[q]] = disease_test_final  
    X_disease_test=data_disease_test.iloc[:,0:100]
    Y_disease_test=data_disease_test.iloc[:,100]
    


KeyboardInterrupt: 

In [12]:
## Save dataframe in xlsx file

import openpyxl
import pandas as pd
data_disease.to_excel('/users/emily/Documents/bertdata/ckd_train.xlsx')
data_disease_test.to_excel('/users/emily/Documents/bertdata/ckd_test.xlsx')

In [1156]:
data_disease

,0,1,2,3,4,5,6,7,8,9,...,91,92,93,94,95,96,97,98,99,d_041
0,-0.534302,-0.420029,1.452468,0.902182,0.487487,-0.914342,1.965251,-0.716631,-0.209050,-0.486976,...,-1.063141,-0.087446,-0.986776,0.967836,0.510392,0.576535,-0.860647,-0.034192,1.486082,0
1,-0.452099,-0.451036,1.129187,1.001932,0.772536,-1.072328,2.183929,-0.731648,-0.102881,-0.513905,...,-1.069379,-0.172302,-0.893126,0.697743,0.374507,0.721090,-0.740164,0.278405,1.628079,0
2,-0.594293,-0.463745,1.134149,0.919339,0.656160,-0.940995,2.080239,-0.794362,-0.192942,-0.481029,...,-0.908677,-0.024950,-1.234947,0.863832,0.417670,0.642738,-0.774727,0.271203,1.635183,0
3,-0.284180,-0.561527,1.405724,1.155246,0.750640,-0.927104,2.069635,-0.643210,-0.085499,-0.610867,...,-0.805959,-0.159184,-1.098702,0.855063,0.745711,0.647041,-0.633737,0.091449,1.506231,0
4,-0.383401,-0.451700,1.259487,0.971805,0.702151,-1.134932,1.895641,-0.636480,-0.090393,-0.486270,...,-0.990229,-0.263319,-1.044072,0.664613,0.388766,0.737661,-0.872071,0.270422,1.589280,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3943,-0.499058,-0.392840,1.194918,0.988054,0.690440,-1.077701,2.143912,-0.581844,-0.049899,-0.422332,...,-1.145476,-0.118901,-1.063352,0.615027,0.576451,0.706793,-0.699817,0.317686,1.579741,0
3944,-0.411873,-0.469769,1.265496,0.986811,0.932690,-1.067300,2.038012,-0.609306,-0.143789,-0.426354,...,-1.081060,-0.098966,-0.884216,0.821535,0.458181,0.704205,-0.706756,0.171911,1.481992,0
3945,-0.369983,-0.479296,1.324836,1.105537,0.810062,-1.116255,2.091126,-0.757863,-0.016157,-0.501653,...,-1.147541,-0.213565,-1.005071,0.689409,0.580894,0.625584,-0.794975,0.181607,1.598699,0
3946,-0.338963,-0.537879,1.275318,1.047665,0.675592,-1.004440,2.009119,-0.677861,-0.152214,-0.522556,...,-1.092134,-0.199042,-1.018013,0.738599,0.396954,0.632930,-0.791577,0.108273,1.436002,0


### Build prediction models and record results (LR and FFN)

In [1145]:
### Logistic regression

searchCV_lasso = LogisticRegressionCV(Cs=list(np.power(10.0, np.arange(-10, 10))),penalty='l1',
cv=5,random_state=777,fit_intercept=True,solver='liblinear',max_iter = 10000,scoring="f1")
lasso_fit_disease = searchCV_lasso.fit(X_train_disease,Y_train_disease)
print(lasso_fit_disease.C_)
lasso_prob_disease = lasso_fit_disease.predict_proba(X_train_disease)[:,1]

lasso_prob_disease = lasso_fit_disease.predict_proba(X_disease_test)[:,1]
auc_disease_lasso = metrics.roc_auc_score(Y_disease_test, lasso_prob_disease)
print('AUC disease Dys:,',auc_disease_lasso)
print('Y_disease_test1 score:', f1_score(Y_disease_test, lasso_fit_disease.predict(X_disease_test)))



[1000000.]
AUC disease Dys:, 0.6259877720071001
Y_disease_test1 score: 0.009523809523809523


In [1165]:
### Feed forward neural network

tr_dataframe = data_disease
te_dataframe = data_disease_test


print("Fineshed\n")

print("Compressing data..."),
tr_X = tr_dataframe.iloc[:, :-1].values
tr_Y = tr_dataframe.iloc[:, -1].values
te_X = te_dataframe.iloc[:, :-1].values
te_Y = te_dataframe.iloc[:, -1].values

Max = tr_X.max(axis=0)
Min = tr_X.min(axis=0)
tr_scale_X = (tr_X - Min)/(Max - Min) * 0.8 + 0.1
te_scale_X = (te_X - Min)/(Max - Min) * 0.8 + 0.1

print("Finished\n")
# In[6]:


def baseline_model(units, act):
    model = Sequential()
    model.add(Dense(units, input_dim=len(tr_X[0]),activation=act))
    model.add(Dense(20,activation = act))
    model.add(Dense(1))

    model.compile(loss='mean_squared_error', optimizer='adam')
    return model



# In[8]:

from tensorflow.keras.callbacks import EarlyStopping
print("Parameter optimization..."),


seed = 7
np.random.seed(seed)

grid_list_units = range(50,60)
grid_list_activation = ['relu','sigmoid']

grid_dict = {'units':grid_list_units, 'act':grid_list_activation}
grid_r2_score = make_scorer(r2_score, greater_is_better=True)
grid_cv = KFold(n_splits=5, shuffle=True, random_state=seed)

#earlystop = EarlyStopping(monitor='val_loss', patience=2, verbose=0, mode='auto')

grid_DL = KerasRegressor(build_fn = baseline_model,batch_size = 300, nb_epoch=1000, verbose = 0)
Grid = GridSearchCV(grid_DL, grid_dict, scoring=grid_r2_score, cv=grid_cv)
Grid.fit(tr_scale_X,tr_Y)

print("Fineshed\n")


# In[31]:

print("\nBest units=%f\nBest activaiton=%s" 
      %(Grid.best_params_['units'], Grid.best_params_['act']))


# In[10]:

from math import sqrt
def model_valuation(y_true, y_pred):
    print("r=%.2f, mae=%.2f, mse=%.2f"
          % (sqrt(r2_score(y_true, y_pred)), mean_absolute_error(y_true, y_pred),
             mean_squared_error(y_true, y_pred)))


# model_DL = KerasRegressor(build_fn = baseline_model(units = Grid.best_params_['units'], act = Grid.best_params_['act']), nb_epoch=2000, batch_size=300,verbose = 0)

model_DL = Sequential()
model_DL.add(Dense(Grid.best_params_['units'], input_dim=len(tr_X[0]),activation=Grid.best_params_['act']))
model_DL.add(Dense(20,activation = Grid.best_params_['act']))
model_DL.add(Dense(1))
model_DL.compile(loss='mean_squared_error', optimizer='adam')



# In[ ]:




# In[32]:


model_DL.fit(tr_scale_X,tr_Y,batch_size = 300, nb_epoch=1000, verbose = 0)
tr_predict_Y = model_DL.predict(tr_scale_X)
te_predict_Y = model_DL.predict(te_scale_X)    



yhat_classes = model_DL.predict_classes(te_scale_X, verbose=0)
f1 = f1_score(te_Y, yhat_classes)
print('F1 score: %f' % f1)

# ROC AUC
auc = metrics.roc_auc_score(te_Y, te_predict_Y)
print('ROC AUC: %f' % auc)




Fineshed

Compressing data...
Finished

Parameter optimization...


KeyboardInterrupt: 